In [23]:
import string
import random
import pickle
import pandas as pd

In [24]:
# Load  the tsv file
with open('../data/tsv/boarding_pass.v1.tsv',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [25]:
# convert the data into a DataFrame
data = list(map(lambda x:x.split('\t'),text.split('\n')))
df = pd.DataFrame(data[1:],columns=data[0])
df.head(10)

,id,text,tag
0,boarding_pass-1.png,ETK-305A6746F,B-TNUMBER
1,boarding_pass-1.png,Passenger,O
2,boarding_pass-1.png,Name,O
3,boarding_pass-1.png,Carrier,O
4,boarding_pass-1.png,Flight,O
5,boarding_pass-1.png,No.,O
6,boarding_pass-1.png,Class,O
7,boarding_pass-1.png,Passenger,O
8,boarding_pass-1.png,Name,O
9,boarding_pass-1.png,_,O


In [26]:
df.shape

(1945, 3)

In [27]:
# Funtion for text cleaning
def text_cleaning(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    text_whitout_whitespace = text.translate(tableWhitespace)
    text_whitout_punctuation = str(text_whitout_whitespace.translate(tablePunctuation))
    return text_whitout_punctuation

In [28]:
# Apply the text cleaning
df['text'] = df['text'].apply(text_cleaning)
dataClean = df.query("text != '' ")
dataClean = dataClean.dropna()
dataClean.head(10)

,id,text,tag
0,boarding_pass-1.png,etk-305a6746f,B-TNUMBER
1,boarding_pass-1.png,passenger,O
2,boarding_pass-1.png,name,O
3,boarding_pass-1.png,carrier,O
4,boarding_pass-1.png,flight,O
5,boarding_pass-1.png,no.,O
6,boarding_pass-1.png,class,O
7,boarding_pass-1.png,passenger,O
8,boarding_pass-1.png,name,O
9,boarding_pass-1.png,_,O


### Convert the data into spacy format

In [29]:
group = dataClean.groupby(by='id')
all_boarding_pass = group.groups.keys()
all_boarding_pass

dict_keys(['boarding_pass-1.png', 'boarding_pass-10.png', 'boarding_pass-11.png', 'boarding_pass-12.png', 'boarding_pass-13.png', 'boarding_pass-14.png', 'boarding_pass-15.png', 'boarding_pass-16.png', 'boarding_pass-17.png', 'boarding_pass-18.png', 'boarding_pass-19.png', 'boarding_pass-2.png', 'boarding_pass-20.png', 'boarding_pass-21.png', 'boarding_pass-22.png', 'boarding_pass-23.png', 'boarding_pass-24.png', 'boarding_pass-25.png', 'boarding_pass-26.png', 'boarding_pass-27.png', 'boarding_pass-28.png', 'boarding_pass-29.png', 'boarding_pass-3.png', 'boarding_pass-30.png', 'boarding_pass-4.png', 'boarding_pass-5.png', 'boarding_pass-6.png', 'boarding_pass-7.png', 'boarding_pass-8.png', 'boarding_pass-9.png'])

In [30]:
from spacy.util import get_words_and_spaces

In [31]:
AllBoardingPassData = []
for boarding_pass in all_boarding_pass:
    BoardingPassData = []
    grouparray = group.get_group(boarding_pass)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    BoardingPassData = (content,annotations)
    AllBoardingPassData.append(BoardingPassData)

### Split the Data into Training and Testing Set

In [32]:
len(AllBoardingPassData)

30

In [33]:
random.shuffle(AllBoardingPassData)
TrainData = AllBoardingPassData[:25]
TestData = AllBoardingPassData[25:]

In [34]:
pickle.dump(TrainData,open('../data/pickle/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('../data/pickle/TestData.pickle',mode='wb'))